# Prepare data for modeling

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import janitor
from pathlib import Path
import re

In [3]:
data_dir = Path('../data')
save_dir = Path('../modeling_data')

## Select cell lines

In [5]:
sample_info = pd.read_csv(save_dir / 'sample_info.csv')

In [6]:
sample_info.head()

,depmap_id,stripped_cell_line_name,ccle_name,sex,cas9_activity,primary_or_metastasis,primary_disease,subtype,lineage,lineage_subtype
0,ACH-000001,NIHOVCAR3,NIHOVCAR3_OVARY,Female,NaN,Metastasis,Ovarian Cancer,"Adenocarcinoma, high grade serous",ovary,ovary_adenocarcinoma
1,ACH-000002,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Female,NaN,Primary,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",blood,AML
2,ACH-000003,CACO2,CACO2_LARGE_INTESTINE,Male,NaN,NaN,Colon/Colorectal Cancer,Adenocarcinoma,colorectal,colorectal_adenocarcinoma
3,ACH-000004,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Male,47.6,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",blood,AML
4,ACH-000005,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Male,13.4,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",blood,AML


In [23]:
def show_counts(df, col):
    return df[[col, 'depmap_id']] \
        .drop_duplicates() \
        .groupby(col) \
        .count() \
        .sort_values('depmap_id', ascending=False)

In [24]:
show_counts(sample_info, 'primary_disease')

,depmap_id
primary_disease,
Lung Cancer,273
Leukemia,132
Skin Cancer,113
Lymphoma,109
Brain Cancer,107
Colon/Colorectal Cancer,83
Breast Cancer,82
Head and Neck Cancer,76
Bone Cancer,75


In [31]:
show_counts(
    sample_info[sample_info.primary_disease == 'Colon/Colorectal Cancer'],
    'subtype'
)

,depmap_id
subtype,
Adenocarcinoma,73
Colorectal Carcinoma,2
"Adenocarcinoma, mucinous",1
"Adenocarcinoma, papillotubular",1
Caecum Adenocarcinoma,1
Colon Adenocarcinoma,1
Colon Carcinoma,1


In [32]:
show_counts(
    sample_info[sample_info.primary_disease == 'Colon/Colorectal Cancer'],
    'sex'
)

,depmap_id
sex,
Male,42
Female,27
Unknown,14


In [33]:
show_counts(
    sample_info[sample_info.primary_disease == 'Colon/Colorectal Cancer'], 
    'primary_or_metastasis'
)

,depmap_id
primary_or_metastasis,
Primary,43
Metastasis,22
